In [1]:
from IPython.display import Image, HTML, display, IFrame

<center> 
<h1> CaNPAN Projects: Nova </h1>
<h2> Introduction </h2>
</center>

<center> 
<h2> General Comments on CaNPAN Projects </h2>
</center>

The main goal of the CaNPAN projects is to provide Canadian nuclear physicists and stellar astrophysicists with computational tools that will help them to collaborate on identifying and solving problems of mutual interests.
Most of these problems are about nuclear reactions going in stars and leading to the production of chemical elements and their isotopes in the Universe. To quicker reach the goal, for each project
* I will first briefly describe a stellar site where nuclear reactions you want to study are likely to play an important role, then 

* I will demonstrate how the evolution of thermodynamic and mixing properties of this site can be simulated with the stellar evolution code MESA and

* I will also explain how to use results of a MESA simulation for one- and multi-zone post-processing nucleosynthesis computations with the NuGrid codes ppn and mppnp, and, finally,

* I will show how to set up and run the NuGrid post-processing nucleosynthesis computatons for Monte Carlo reaction rate sensitivity and uncertainty studies.

<center>
<h4>Nova explosions occur in close binary systems on white dwarfs accreting H-rich material from their main-sequence, sub-giant or red-giant companions (a picture from slideplayer.com)</h4>
<img src="img/NovaBinarySystem.jpg", width=1000>
</center>

In [2]:
# on an astrohub server use ipympl that enables interactive features of 
# matplotlib in a Jupyter notebook and in JupyterLab
%pylab ipympl  

# for a jupyter classic notebook use (uncomment)
#%pylab nbagg

# astronomical and physical constants in SI units
from astropy import constants as const
from astropy import units as u

Msun = (const.M_sun).value
Rsun = (const.R_sun).value
Lsun = (const.L_sun).value
GN = (const.G).value  # Newton's gravitational constant 
sigma = (const.sigma_sb).value
print ('Msun =',Msun,(const.M_sun).unit)
print ('Rsun =',Rsun,(const.R_sun).unit)
print ('Lsun =',Lsun,(const.L_sun).unit)
print ("Newton's G =",GN,(const.G).unit)
print ("Stefan-Boltzmann constant =",sigma,(const.sigma_sb).unit)

Populating the interactive namespace from numpy and matplotlib
Msun = 1.988409870698051e+30 kg
Rsun = 695700000.0 m
Lsun = 3.828e+26 W
Newton's G = 6.6743e-11 m3 / (kg s2)
Stefan-Boltzmann constant = 5.6703744191844314e-08 W / (K4 m2)


In [3]:
# calculate Roche lobe (RL) radii of a WD and its companion in solar radii

# parameters of a binary system
m1 = 1.0 # mass of the WD in solar masses
m2 = 1.5 # mass of the companion in solar masses
P = 4 # orbital period of the WD in hours, 
# for orbital periods of nova systems 
# see Table 1 in https://ui.adsabs.harvard.edu/abs/1997A%26A...322..807D/abstract

q1 = m1/m2 # WD to companion mass ratio

# semi-major axis in solar radii from Kepler's law
aRsun = (GN*Msun*3600**2/(4*pi**2*Rsun**3))**(1./3.)*P**(2./3.)*(m2*(1+q1))**(1./3.) 

# I use equation (2) from https://ui.adsabs.harvard.edu/abs/1983ApJ...268..368E/abstract
# to calculate RL radius of WD in units of semi-major axis
RLa1 = 0.49*q1**(2./3.)/(0.6*q1**(2./3.)+log(1+q1**(1./3.)))

# RL radius of WD in solar radii
RL1Rsun = RLa1*aRsun

q2 = m2/m1 # companion to WD mass ratio

# RL radius of WD companion in units of semi-major axis
RLa2 = 0.49*q2**(2./3.)/(0.6*q2**(2./3.)+log(1+q2**(1./3.)))

# RL radius of WD companion in solar radii
RL2Rsun = RLa2*aRsun 

print ("a/R_sun = %5.3f" % aRsun)
print ("RL_WD/R_sun = %5.3f" % RL1Rsun,"\nRL_comp/R_sun = %5.3f" % RL2Rsun)

a/R_sun = 1.730
RL_WD/R_sun = 0.596 
RL_comp/R_sun = 0.717


<center> 
<h4>Actual and possible observables of nova explosion and nucleosynthesis</h4>        
</center>

* suddenly, a new (nova) star "appears" in the sky, which is caused by an increase of WD's luminosity
from $10^{-2}\,L_\odot$ to $10^4\,L_\odot$, i.e. by 15 magnitudes (3 to 5 novae are observed per year, although 30 to 50 novae are expected to explode in the Galaxy every year);

* optical, ultraviolet, and X-ray light curves from deepening nova photosphere, as the ejected nova envelope expands, and infrared emission from forming dust (observed);

* elemental abundances in nova ejecta determined using their optical, UV, and IR emission-line spectra (measured, see Table 2 below from https://ui.adsabs.harvard.edu/abs/1998PASP..110....3G/abstract); 

* $\gamma$ lines from electron-positron annihilation at 511 keV with adjacent continuum at lower energies cut-off
at 20-30 keV; $\gamma$ lines at 478 keV, 1275 keV, and 1.809 MeV from decaying $^{7}$Be (its lifetime $\tau = 77$ days depends on the temperature and density), $^{22}$Na ($\tau = 3.75$ yr), and $^{26}$Al ($\tau = 1.03$ Myr), respectively (have not been observed yet);

* peculiar abundance ratios of CNO and intermediate-mass isotopes produced in non-equilibrium ($\beta^+$-decay limited) CNO-, NeNa-, MgAl-cycles, and beyound, up to $A<40$ (measured in presolar grains from meteoritic samples). 

<center>
<h4>HST images of Nova Cygni 1992 (V1974) taken 7 months apart in 1993-1994 (https://en.wikipedia.org/wiki/V1974_Cygni)</h4>
<img src="img/classicalnovae1.jpg", width=800>
</center>

<center>
<h4>In this project, we focus on H-burning nucleosynthesis in 1D hydrostatic nova models, rather than on nova photometry or spectroscopy at different color bands or wavelengths</h4>
</center>

The H-rich material accreted by the white dwarf is being compressed by the gravitational force and, as a result of this compression, it is getting heated up until H is ignited leading to a thermonuclear explosion. A large amount of heat released by this runaway H burning at the base of the accreted envelope triggers convection in the envelope and causes its expansion. It is interesting that a similar sequence of events, namely air intake, its compression, fuel
injection and combustion, finally followed by exaust of burning products, takes place in a four-stroke diesel engine (see the animation below). The similarity between the nova and diesel engine duty cycles is enhanced by the fact that a successfull nova explosion also needs an injection (mixing) of CO- or ONe-rich material from an underlying CO or ONe WD that serves as catalyst for H burning in the CNO cycle.

<center>
<h4>A mechanical analogue of novae is a four-stroke diesel engine</h4>     
<img src="img/diesel_v5.gif", width=800>
</center>

In [4]:
HTML('<iframe width="1100" height="800" src="https://www.youtube.com/embed/6J468JHtPuE" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:701: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


<center> 
<h4>The strength of a nova explosion (the maximum temperature of H burning and the envelope expansion speed) is amplified with</h4>        
</center>

* an increase of the WD mass $M_\mathrm{WD}\leq 1.38\,M_\odot$ (a WD with a higher mass has a smaller radius, hence a deeper gravitational potential well, therefore the accreted material is stronger compressed); this also reduces the total accreted mass before the nova explosion is triggered

* a decrease of the WD central temperature $T_\mathrm{WD}$ (or luminosity), which slows down the heating up of the compressed accreted material; this results in a higher accreted mass

* a decrease of the mass accretion rate $\frac{dM_\mathrm{acc}}{dt}$, which reduces the gravothermal energy release rate, thus also leading to a a higher accreted mass 

* an enrichment of the accreted envelope in C and O that are most likely dredged up from the underlying WD by convective boundary mixing (the Kelvin-Helmholtz instability) that is also needed to erase the He-buffer layer from the surface of a new-born WD (see the movie http://www.fen.upc.edu/users/jjose/images/nova1-e.mp4 from the paper https://ui.adsabs.harvard.edu/abs/2016A%26A...595A..28C/abstract)

<center>
<img src="img/Gehrz1998Table2.png", width=1200>
</center>

<center>
<h4>From https://ui.adsabs.harvard.edu/abs/2006NuPhA.777..550J/abstract</h4>
<img src="img/JoseRev2006Fig1.png", width=800>
</center>

<center>
<h4>From https://ui.adsabs.harvard.edu/abs/2006NuPhA.777..550J/abstract</h4>
<img src="img/JoseRev2006Fig4.png", width=800>
</center>

<center>
<h4>From https://ui.adsabs.harvard.edu/abs/2006NuPhA.777..550J/abstract</h4>
<img src="img/JoseRev2006Fig5.png", width=800>
</center>